In [ ]:
import os
import glob
import requests
import pyproj
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List
from string import ascii_uppercase


from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook, output_file
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.models import LassoSelectTool, HoverTool, TapTool, LabelSet
from bokeh.models import LinearColorMapper, ColorBar, LogColorMapper
from bokeh.models import WMTSTileSource
from bokeh.tile_providers import CARTODBPOSITRON, CARTODBPOSITRON_RETINA, STAMEN_TERRAIN
from bokeh.palettes import Viridis256, Magma256, Inferno256

output_notebook()


%matplotlib inline
%config IPCompleter.use_jedi = False

In [ ]:
from IPython.display import Image
Image('jakevdp_landscape.png')

In [ ]:
## hacer el mapa de estaciones poco a poco
# ver como hacer el streaming o poner un slider

## Bokeh básico

In [ ]:
# tools="pan,wheel_zoom,box_zoom,reset"

p = figure(plot_width=400, plot_height=400)

x = [1, 2, 3, 4, 5]
y = [2, 5, 8, 2, 7]

r = p.circle(x, y, size=20, fill_alpha=0.5)

show(p)

In [ ]:
p = figure(plot_width=400, plot_height=400)

data = {'x': [1, 2, 3, 4, 5],
        'y': [2, 5, 8, 2, 7]}

source = ColumnDataSource(data)

r = p.circle('x', 'y', size=20,
             source=source,
             selection_color="firebrick",
             nonselection_fill_alpha=0.2,
             nonselection_fill_color="grey",
             nonselection_line_color="firebrick",
             nonselection_line_alpha=1.0)

p.add_tools(LassoSelectTool())
p.add_tools(TapTool())

# hover 

tooltips = [
    ('x', '@x'),
    ('y', '@y')
]

p.add_tools(HoverTool(tooltips=tooltips, mode='mouse'))

show(p)

In [ ]:
p = figure(plot_width=600, plot_height=500)

data = {'x': [1, 2, 3, 4, 5],
        'y': [2, 5, 8, 2, 7],
        'nombre': [l for l in ascii_uppercase[0:5]]}

source = ColumnDataSource(data)

r = p.circle('x', 'y', size=20, source=source)

tooltips = [
    ('x', '@x'),
    ('y', '@y'),
    ('ID', '@nombre')
]

p.add_tools(HoverTool(tooltips=tooltips, mode='mouse'))
p.xaxis.axis_label = 'x (m)'
p.yaxis.axis_label = 'y (m)'

labels = LabelSet(x='x', y='y', text='nombre',
                  x_offset=-5, y_offset=10, source=source)

p.add_layout(labels)

show(p)

In [ ]:
p = figure(plot_width=600, plot_height=500, x_range=(0, 10), y_range=(0, 15))

data = {'x': [1, 2, 3, 4, 5],
        'y': [2, 5, 8, 2, 7],
        'z': [20, 12, 50, 42, 29],
        'nombre': [l for l in ascii_uppercase[0:5]]}

source = ColumnDataSource(data)

# colorbar
color_mapper = LinearColorMapper(palette=Viridis256,
                                 low=min(source.data['z']),
                                 high=max(source.data['z']))

r = p.circle('x', 'y', size=20, source=source,
             color={'field': 'z', 'transform': color_mapper})

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=5, location=(0, 0), title='z')
p.add_layout(color_bar, 'left')

# hover
tooltips = [
    ('x', '@x'),
    ('y', '@y'),
    ('ID', '@nombre')
]

p.add_tools(HoverTool(tooltips=tooltips, mode='mouse'))
p.xaxis.axis_label = 'x (m)'
p.yaxis.axis_label = 'y (m)'

# label set
labels = LabelSet(x='x', y='y', text='nombre',
                  x_offset=-5, y_offset=10, source=source)
p.add_layout(labels)

show(p)

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
## Ejercicio. Gráfica de las estaciones coloreadas por número de Arribos

In [ ]:
df_estaciones = pd.read_csv('data/estaciones_ecobici.csv')
df_estaciones.head()

In [ ]:
df_viajes = pd.read_parquet('data/viajes_ecobici.parquet', engine='pyarrow')
df_arribos = df_viajes.groupby('Ciclo_Estacion_Arribo', as_index=False).Bici.count()
df_arribos = df_arribos.rename(columns={'Ciclo_Estacion_Arribo': 'estacion_id',
                                        'Bici': 'num_arribos'})
df_arribos.head()

In [ ]:
# 

## Agregar tiles

In [ ]:
df_estaciones = pd.read_csv('data/estaciones_ecobici.csv')

df_viajes = pd.read_parquet('data/viajes_ecobici.parquet', engine='pyarrow')
df_arribos = df_viajes.groupby('Ciclo_Estacion_Arribo', as_index=False).Bici.count()
df_arribos = df_arribos.rename(columns={'Ciclo_Estacion_Arribo': 'estacion_id',
                                        'Bici': 'num_arribos'})


df_mapa = pd.merge(df_estaciones, df_arribos, on='estacion_id', how='inner')
df_mapa.head()

In [ ]:
def wgs84_to_web_mercator(latitudes,
                          longitudes,
                          in_proj: str = 'epsg:4326',
                          out_proj: str = 'epsg:3857'):
    """Utility function to transform latitude and longitude values 
    to web marcator values"""
    output =  pyproj.transform(pyproj.Proj(init=in_proj),
                               pyproj.Proj(init=out_proj),
                               longitudes, latitudes)
    return output[::-1]

lat, lon = 19.420487, -99.173006
y, x = wgs84_to_web_mercator(lat, lon)

## Interacciones (ipywidgets)

In [ ]:
import ipywidgets
from ipywidgets import interact

In [ ]:
slider = ipywidgets.IntSlider()
slider

In [ ]:
slider.value = 90

In [ ]:
texto = ipywidgets.IntText()
texto

In [ ]:
mylink = ipywidgets.jslink((slider, 'value'), (texto, 'value'))

In [ ]:
valor = 0

In [ ]:
def funcion(x):
    return x // 2

interact(funcion, x=(0, 10));

In [ ]:
def funcion(x):
    return x

interact(funcion, x=False);

In [ ]:
def saludar(x):
    if x == 'ES':
        return 'Hola'
    elif x == 'EN':
        return 'Hello'
    else:
        return b'\xF0\x9F\x98\x90'.decode('utf-8')


interact(saludar, x=['ES', 'EN', 'FR']);

In [ ]:
# x={'ES': 'Hola', 'EN': 'Hello', 'FR': b'\xF0\x9F\x98\x90'.decode('utf-8')}

### Handles

In [ ]:
p = figure(plot_width=500, plot_height=400, x_range=(0, 8), y_range=(0, 10))

data = {'x': [1, 2, 3, 4, 5],
        'y': [2, 5, 8, 2, 7],
        'z': [20, 12, 50, 42, 29]}

source = ColumnDataSource(data)

r = p.circle('x', 'y', size=20, source=source)

t = show(p, notebook_handle=True)

In [ ]:
r.glyph.fill_alpha = 0.5

push_notebook(handle=t)

In [ ]:
# source.data

### Bokeh + ipywidgets

In [ ]:
x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

p = figure(plot_height=300, plot_width=600, y_range=(-5,5))
r = p.line(x, y, color='purple', line_width=3)

In [ ]:
def update(f, w=1, A=1, phi=0):
    if f == "sin":
        func = np.sin
    elif f == "cos":
        func = np.cos
    elif f == "tan":
        func = np.tan
    r.data_source.data['y'] = A * func(w * r.data_source.data['x'] + phi)
    push_notebook(handle=t)


In [ ]:
t = show(p, notebook_handle=True)

In [ ]:
interact(update, f=["sin", "cos", "tan"], w=(0, 100), A=(1, 5), phi=(0, 20, 0.1));

## Ejercicio: Mapa con slider para ver los arribos por hora (7-23 horas)

### Visualización con muchos datos

* http://datashader.readthedocs.io/en/latest/
* http://vaex.astro.rug.nl

### Hover serie de tiempo

In [ ]:
df_ts = df_viajes.set_index('Fecha_Retiro').resample('D').Bici.count().reset_index()

source = ColumnDataSource(df_ts)

p = figure(width=900, height=400, x_axis_type="datetime")

p.line(x='Fecha_Retiro', y='Bici', line_width=1, color='gray', source=source)

cr = p.circle(x='Fecha_Retiro', y='Bici', size=20, source=source,
              fill_color="grey", hover_fill_color="firebrick",
              fill_alpha=0.05, hover_alpha=0.3,
              line_color=None, hover_line_color="white")

p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='vline'))

show(p)